<a href="https://colab.research.google.com/github/LeVraisPeg/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Vérifie si le notebook est exécuté sur Google Colab
if "COLAB_GPU" in os.environ or "COLAB_TPU_ADDR" in os.environ:
    print("Exécution sur Google Colab")

    # Clone du dépôt
    !git clone https://github.com/vincentmartin/tp-rag-student-version.git

    # Se placer dans le dossier du projet
    %cd tp-rag-student-version

    # Installation des dépendances
    !pip install -r requirements.txt
else:
    print("Pas sur Google Colab")


Exécution sur Google Colab
fatal: destination path 'tp-rag-student-version' already exists and is not an empty directory.
/content/tp-rag-student-version
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
from pathlib import Path

DATA_DIR = Path("data")

for p in DATA_DIR.rglob("*"):
    print(p)


data/patents
data/latex
data/autres_articles
data/arxiv
data/patents/BANDAGE.txt
data/patents/METHOD AND DEVICE FOR MANUFACTURING CONSTRUCTION BLOCKS FROM A HYDRAULIC BINDER SUCH AS PLASTER, AN INERT FILLER SUCH AS SAND, AND WATER.txt
data/patents/Electromagnetic radiation monitor.txt
data/patents/Method for deriving character features in a character recognition system.txt
data/patents/APPARATUS FOR THE MEASUREMENT OF ATRIAL PRESSURE.txt
data/patents/ATMSPOS BASED ELECTRONIC MAIL SYSTEM.txt
data/patents/MICROWAVE TURNTABLE CONVECTION HEATER.txt
data/patents/SPECIFIC DNA SEQUENCES OF A NEMATODE WHICH CAN BE USED FOR THE DIAGNOSIS OF INFECTION WITH THE NEMATODE.txt
data/patents/CONDOM FOR ORAL-GENITAL USE.txt
data/patents/Wear component and method of making same.txt
data/patents/PHARMACEUTICAL COMPOSITIONS OF GALLIUM COMPLEXES OF 3-HYDROXY-4-PYRONES.txt
data/patents/SELECTION OF RIBOZYMES THAT EFFICIENTLY CLEAVE TARGET RNA.txt
data/patents/DEVICE FOR FORMATION OF A FILM ON THE WALLS OF H

In [3]:
!pip install langchain langchain-community chromadb sentence-transformers
!pip install langchain-text-splitters
!pip install pypdf


In [4]:
!pip -q install --upgrade \
  langchain langchain-core langchain-community \
  langchain-huggingface langchain-chroma \
  chromadb sentence-transformers pypdf


#Etape 1. - Indexation des documents

##Exercice 1 : indexation

In [5]:
import os
from pathlib import Path
from typing import List

import torch
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

DATA_DIR = Path("data")
PERSIST_DIR = "chroma_db"
COLLECTION_NAME = "tp_rag"
EMBED_MODEL = "intfloat/multilingual-e5-base"


## Nettoyage des fichiers LaTeX (.tex)

Le corpus contient des documents au format LaTeX. Avant l’indexation dans une base vectorielle, on réalise un **pré-traitement** pour éviter d’indexer du bruit :

- suppression des lignes commentées (`% ...`)
- suppression du préambule LaTeX (avant `\begin{document}`)
- suppression de tout ce qui suit `\end{document}`
- suppression de commandes fréquentes du "front-matter" (titre, auteurs, packages…)
- normalisation des espaces et retours à la ligne

Objectif : **ne conserver que le contenu textuel utile** pour améliorer la qualité des embeddings et donc du retrieval.


In [6]:
import re

LATEX_FRONTMATTER_CMDS = [
    "Title", "TitleCitation", "author", "affil", "date", "abstract",
    "keywords", "maketitle", "documentclass", "usepackage"
]

def clean_latex(text: str) -> str:
    if not text:
        return ""

    # Enlever lignes commentées
    text = "\n".join([ln for ln in text.splitlines() if not ln.lstrip().startswith("%")])

    # Si \begin{document} existe, on garde seulement après
    m = re.search(r"\\begin\s*\{\s*document\s*\}", text, flags=re.IGNORECASE)
    if m:
        text = text[m.end():]

    # Enlever tout après \end{document} si présent
    text = re.split(r"\\end\s*\{\s*document\s*\}", text, flags=re.IGNORECASE)[0]


    #clean
    for cmd in LATEX_FRONTMATTER_CMDS:
        text = re.sub(rf"\\{cmd}\s*\{{.*?\}}", " ", text, flags=re.IGNORECASE | re.DOTALL)


    text = re.sub(r"\n{3,}", "\n\n", text)
    text = re.sub(r"[ \t]{2,}", " ", text)

    return text.strip()


## Chargement du corpus (TXT/MD, LaTeX, PDF)

Le corpus documentaire est stocké dans le dossier `data/` et contient plusieurs formats.
Pour pouvoir indexer tous les documents de manière uniforme, on met en place une fonction de chargement qui :

- parcourt récursivement le dossier (`rglob("*")`)
- détecte le type de fichier via l’extension
- charge le contenu avec un loader LangChain adapté :
  - `.txt` / `.md` : `TextLoader`
  - `.tex` : `TextLoader` + nettoyage via `clean_latex()` (suppression du bruit LaTeX)
  - `.pdf` : `PyPDFLoader`

Chaque fichier chargé est converti en objet `Document` (LangChain), qui contient :
- `page_content` : le texte
- `metadata` : des informations utiles (notamment `source`, c.-à-d. le chemin du fichier), très utiles ensuite pour tracer les réponses du chatbot.


In [7]:
def load_all_documents(root_dir: Path) -> List[Document]:
    docs: List[Document] = []

    for path in root_dir.rglob("*"):
        if not path.is_file():
            continue

        suffix = path.suffix.lower()

        # TXT
        if suffix in [".txt", ".md"]:
            loader = TextLoader(str(path), encoding="utf-8")
            docs.extend(loader.load())

        # TEX
        elif suffix == ".tex":
            loader = TextLoader(str(path), encoding="utf-8")
            loaded = loader.load()
            for d in loaded:
                d.page_content = clean_latex(d.page_content)
            docs.extend(loaded)

        # PDF
        elif suffix == ".pdf":
            loader = PyPDFLoader(str(path))
            docs.extend(loader.load())

        else:
            # formats ignorés (.bib, etc.)
            pass

    return docs


documents = load_all_documents(DATA_DIR)
from collections import Counter

sources = [str(d.metadata.get("source","")) for d in documents]
exts = [Path(s).suffix.lower() for s in sources]
print("Extensions chargées:", Counter(exts))


print(f"Nombre total de documents chargés : {len(documents)}")
print("Exemple source :", documents[0].metadata.get("source") if documents else "N/A")


Extensions chargées: Counter({'.pdf': 555, '.txt': 27, '.tex': 9})
Nombre total de documents chargés : 591
Exemple source : data/patents/BANDAGE.txt


## Découpage des documents en paragraphes (chunking)

Avant de calculer les embeddings, les documents doivent être découpés en unités plus petites appelées *chunks*.
Dans ce TP, nous avons choisi un **découpage par paragraphes**, basé sur les doubles retours à la ligne (`\n\n`).

Pour chaque document :
- le texte est découpé en paragraphes
- les paragraphes trop courts (moins de 50 caractères) sont ignorés
- chaque paragraphe valide devient un nouveau `Document` LangChain

Un identifiant `chunk_id` est ajouté dans les métadonnées afin de :
- conserver l’ordre des paragraphes dans le document d’origine
- permettre plus tard de regrouper les chunks provenant d’un même fichier (ex. résumé de document complet)


In [8]:
def split_by_paragraphs(docs: List[Document], min_len: int = 50) -> List[Document]:
    out: List[Document] = []
    for d in docs:
        text = d.page_content or ""
        paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]

        for i, p in enumerate(paragraphs):
            if len(p) < min_len:
                continue
            out.append(Document(
                page_content=p,
                metadata={**d.metadata, "chunk_id": i}
            ))
    return out

paragraph_docs = split_by_paragraphs(documents, min_len=50)
print(f"Nombre total de paragraphes : {len(paragraph_docs)}")


Nombre total de paragraphes : 1914


## Filtrage des paragraphes non pertinents (bruit et bibliographie)

Après le découpage en paragraphes, tous les chunks ne sont pas utiles pour la recherche sémantique.
Certains contiennent uniquement du bruit, par exemple :
- bibliographies et références
- préambule ou structure LaTeX
- paragraphes majoritairement composés de commandes ou de commentaires

Nous mettons donc en place une fonction heuristique `is_bibliography_or_noise` qui permet
d’exclure ces paragraphes avant l’indexation.


In [9]:
import re
BIB_PATTERNS = [
    r"\\bibitem",
    r"\\newblock",
    r"\\begin\{thebibliography\}",
    r"\\end\{thebibliography\}",
]
NOISE_PATTERNS = [
    r"\\begin\{document\}",
    r"\\end\{document\}",
    r"\\documentclass",
    r"\\usepackage",
    r"\\title\{",
    r"\\author\{",
    r"\\maketitle",
]

def is_bibliography_or_noise(text: str) -> bool:
    t = (text or "").strip()
    if not t:
        return True

    # Bibliographie / références
    if any(re.search(p, t) for p in BIB_PATTERNS):
        return True

    # Préambule LaTeX / structure
    if any(re.search(p, t) for p in NOISE_PATTERNS):
        return True


    lines = [ln.strip() for ln in t.splitlines() if ln.strip()]
    if lines:
        comment_ratio = sum(1 for ln in lines if ln.startswith("%")) / len(lines)
        if comment_ratio > 0.6:
            return True


    if t.count("\\") / max(len(t), 1) > 0.03:
        return True

    return False

before = len(paragraph_docs)
docs_for_index = [d for d in paragraph_docs if not is_bibliography_or_noise(d.page_content)]
after = len(docs_for_index)
print(f"Filtrage: {before} -> {after} chunks conservés (supprimés: {before-after})")


Filtrage: 1914 -> 1406 chunks conservés (supprimés: 508)


## Initialisation du modèle d’embeddings et préparation des documents

Avant de créer l’index vectoriel, nous initialisons le modèle d’embeddings
et préparons les documents à indexer.

### Réinitialisation de l’index
Pour éviter toute incohérence liée à un index existant (changement de paramètres,
nouveaux documents, nouveau pré-traitement), le dossier de persistance est supprimé
s’il existe déjà.

### Choix du modèle d’embeddings
Nous utilisons le modèle **`intfloat/multilingual-e5-base`**, qui présente plusieurs avantages :
- représentation sémantique de haute qualité
- support multilingue
- modèle largement utilisé pour la recherche sémantique

Le calcul des embeddings est effectué :
- sur GPU si disponible (`cuda`)
- sinon sur CPU


In [10]:
import shutil, os

# Reset pour éviter de réutiliser un index sale
if os.path.exists(PERSIST_DIR):
    shutil.rmtree(PERSIST_DIR)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device embeddings :", device)

embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},
)



docs_prefixed = [
    Document(page_content="passage: " + d.page_content, metadata=d.metadata)
    for d in docs_for_index
]


Device embeddings : cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Création et remplissage de la base vectorielle (ChromaDB)

Une fois les documents nettoyés, découpés et transformés en embeddings,
nous les indexons dans une base vectorielle afin de permettre la recherche sémantique.

Nous utilisons **ChromaDB** comme vector store pour les raisons suivantes :
- simplicité d’utilisation
- persistance sur disque
- intégration native avec LangChain


In [11]:
CLEAN_COLLECTION_NAME = "tp_rag_clean_v2"



vectorstore_clean = Chroma(
    collection_name=CLEAN_COLLECTION_NAME,
    persist_directory=PERSIST_DIR,
    embedding_function=embeddings,
)

existing = vectorstore_clean._collection.count()
print("Déjà dans l'index clean :", existing)

if existing == 0:
    vectorstore_clean.add_documents(docs_prefixed)
    print("Indexation clean terminée. Total :", vectorstore_clean._collection.count())
else:
    print("Index clean déjà présent.")


Déjà dans l'index clean : 0
Indexation clean terminée. Total : 1406


In [12]:
  query = "query: de quoi parlent les documents ?"
  results = vectorstore_clean.similarity_search(query, k=3)


  for i, d in enumerate(results, 1):
      print(f"\n--- Résultat {i} ---")
      print("Source:", d.metadata.get("source"))
      print(d.page_content[:300])



--- Résultat 1 ---
Source: data/latex/Complex QA & language models hybrid architectures, Survey.tex
passage: \citet{rogersQADatasetExplosion2022} proposes an "evidence format" for the explainable part of a dataset composed of Modality (Unstructured text, Semi-structured text, Structured knowledge, Images, Audio, Video, Other combinations) and Amount of evidence (Single source, Multiple sources, Pa

--- Résultat 2 ---
Source: data/latex/A Survey of Software-Defined Smart Grid Networks, Security Threats and Defense Techniques.tex
passage: \end{abstract}
\begin{IEEEkeywords}
smart grid, software-defined networking, network security, cybersecurity
\end{IEEEkeywords}

--- Résultat 3 ---
Source: data/latex/Reconfigurable Intelligent Surface Assisted Railway Communications A survey.tex
passage: \subsection{Railway environments characteristics} %\textcolor{red}{je trouve qu'il faudrait aller à la ligne mais je ne sais pas comment -> pas besoin, il faut laisser faire latex}


##Exercice 2 : interrogation

## Recherche sémantique dans la base vectorielle

Une fois les documents indexés dans ChromaDB, nous mettons en place une fonction
permettant d’interroger la base à partir d’une requête utilisateur.

Contrairement à une recherche par mots-clés, cette recherche est **sémantique** :
elle repose sur la similarité entre les embeddings de la requête et ceux des documents.


In [13]:
from typing import List, Tuple
from langchain_core.documents import Document
from langchain_chroma import Chroma

def retrieve_documents(
    vectorstore: Chroma,
    query: str,
    k: int = 5
) -> List[Tuple[Document, float]]:
    """
    Interroge la base vectorielle et retourne les documents
    les plus proches de la requête avec leur score associé.
    """

    formatted_query = query if query.startswith("query:") else f"query: {query}"

    results = vectorstore.similarity_search_with_score(
        formatted_query,
        k=k
    )

    return results


In [14]:
query = "railway communications"
results = retrieve_documents(vectorstore_clean, query, k=3)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n--- Résultat {i} ---")
    print(f"Score: {score:.4f}")
    print("Source:", doc.metadata.get("source"))
    print(doc.page_content[:300])



--- Résultat 1 ---
Score: 0.2805
Source: data/latex/Reconfigurable Intelligent Surface Assisted Railway Communications A survey.tex
passage: \begin{IEEEkeywords} RIS, Railway communications, mmWave.

--- Résultat 2 ---
Score: 0.2939
Source: data/latex/Reconfigurable Intelligent Surface Assisted Railway Communications A survey.tex
passage: Considering the capability of RIS to solve the blockage problems in mmWave wireless communications, the use of RIS for railway communications has recently been considered as a promising candidate.

--- Résultat 3 ---
Score: 0.3151
Source: data/latex/Reconfigurable Intelligent Surface Assisted Railway Communications A survey.tex
passage: \section{Conclusion}
\label{sec:conclusion}
This paper presents a survey on RIS-assisted communications for railway applications, particularly in the mmWave band. First, we have defined the RIS concept, explaining its structure, and different types of RISs. A review of the various optimizat


#Etape 2. - RAG

##Exercice 3. : prompt template

In [15]:
from langchain_core.prompts import PromptTemplate

RAG_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Tu es un assistant de Q/R. Tu dois répondre uniquement à partir du CONTEXTE.\n"
        "Si l'information n'est pas présente dans le contexte, dis clairement: "
        "\"Je ne sais pas d'après les documents fournis.\".\n\n"
        "=== CONTEXTE ===\n"
        "{context}\n"
        "=== FIN CONTEXTE ===\n\n"
        "Question: {question}\n"
        "Réponse (en français, claire et synthétique):"
    )
)


##Exercice 4. : chaîne RAG

In [16]:
!apt-get update -qq
!apt-get install -y zstd

!curl -fsSL https://ollama.ai/install.sh | sh


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 90 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [17]:
!nohup ollama serve > ollama.log 2>&1 &


In [18]:
!curl -s http://localhost:11434/api/version || echo "Ollama DOWN"
!ss -ltnp | grep 11434 || echo "port 11434 fermé"


Ollama DOWN
port 11434 fermé


In [19]:
!ollama pull qwen3:8b
!ollama list


NAME        ID              SIZE      MODIFIED               
qwen3:8b    500a1f067a9f    5.2 GB    Less than a second ago    


## Chaîne RAG : génération de réponse augmentée par la recherche

Dans cette étape, nous mettons en œuvre une chaîne complète de **Retrieval Augmented Generation (RAG)**.
Le principe consiste à :
1. rechercher les passages pertinents dans la base vectorielle,
2. construire un contexte textuel à partir de ces passages,
3. fournir ce contexte à un modèle de langage afin de générer une réponse fidèle aux documents.

Le modèle de langage utilisé est **Qwen3**, servi localement via **Ollama**.


In [20]:
from langchain_community.chat_models import ChatOllama
from langchain_core.documents import Document
from typing import List, Tuple

def format_context(docs_with_scores: List[Tuple[Document, float]], max_chars: int = 5000) -> str:
    parts = []
    total = 0
    for doc, score in docs_with_scores:
        src = doc.metadata.get("source", "unknown_source")
        chunk_id = doc.metadata.get("chunk_id", None)
        header = f"[source={src} chunk={chunk_id} score={score:.4f}]"
        text = doc.page_content.replace("passage: ", "").strip()
        block = f"{header}\n{text}\n"
        if total + len(block) > max_chars:
            break
        parts.append(block)
        total += len(block)
    return "\n".join(parts)

llm = ChatOllama(
    model="qwen3:8b",
    temperature=0,
)

def rag_answer(vectorstore, prompt_template, question: str, k: int = 5) -> str:
    # 1) retrieval
    docs_with_scores = retrieve_documents(vectorstore, question, k=k)

    # 2) build context
    context = format_context(docs_with_scores)

    # 3) prompt
    prompt = prompt_template.format(context=context, question=question)

    # 4) LLM
    resp = llm.invoke(prompt)
    return resp.content



/tmp/ipython-input-4133255840.py:20: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [21]:
print(rag_answer(vectorstore_clean, RAG_PROMPT, "De quoi parle le document sur les railway communications ?", k=5))


Le document présente une revue des communications ferroviaires assistées par les surfaces intelligentes réconfigurables (RIS), notamment dans la bande millimétrique (mmWave). Il explore les concepts de RIS, les algorithmes d'optimisation, la résolution du problème de blocage des ondes mmWave, et les applications dans les trains à grande vitesse.


##Exercice 5. : mémoire

## Reformulation des questions et gestion de la mémoire

Ce code met en place un mécanisme de mémoire conversationnelle pour un chatbot RAG.

Il conserve l’historique des échanges entre l’utilisateur et l’assistant et l’utilise pour reformuler
les questions de suivi en questions complètes et autonomes.

Concrètement :
- l’historique de la conversation est converti en texte exploitable par le modèle,
- une question de suivi est reformulée à l’aide de l’historique afin d’en expliciter le sujet,
- la recherche vectorielle est effectuée à partir de cette question reformulée,
- le contexte récupéré et l’historique sont injectés dans le prompt de génération,
- la réponse produite est ajoutée à l’historique pour les tours suivants.

Ce mécanisme permet au chatbot de gérer des échanges multi-tours tout en conservant
une recherche documentaire cohérente à chaque interaction.


In [28]:
from langchain_core.prompts import PromptTemplate

CONDENSE_QUESTION_PROMPT = PromptTemplate(
    input_variables=["chat_history", "question"],
    template=(
        "Tu reçois un HISTORIQUE de conversation et une question de suivi.\n"
        "Réécris la question de suivi en une question autonome, claire et complète, "
        "en français, en intégrant les informations nécessaires depuis l'historique.\n\n"
        "=== HISTORIQUE ===\n"
        "{chat_history}\n"
        "=== FIN HISTORIQUE ===\n\n"
        "Question de suivi: {question}\n\n"
        "Question autonome:"
    )
)

def render_chat_history(history: list, max_turns: int = 8) -> str:
    # history = [{"role":"user","content":...}, {"role":"assistant","content":...}, ...]
    trimmed = history[-2*max_turns:]
    lines = []
    for m in trimmed:
        role = "Utilisateur" if m["role"] == "user" else "Assistant"
        lines.append(f"{role}: {m['content']}")
    return "\n".join(lines)

def rewrite_question(question: str, history: list) -> str:
    chat_history_str = render_chat_history(history)
    prompt = CONDENSE_QUESTION_PROMPT.format(
        chat_history=chat_history_str,
        question=question
    )
    standalone = llm.invoke(prompt).content.strip()
    return standalone

def rag_chat(vectorstore, question: str, history: list, k: int = 5) -> str:
    # Rewrite question si historique non vide
    if history:
        standalone_question = rewrite_question(question, history)
    else:
        standalone_question = question

    # Retrieval sur question autonome
    docs_with_scores = retrieve_documents(vectorstore, standalone_question, k=k)
    context = format_context(docs_with_scores)
    chat_history_str = render_chat_history(history)

    # Generation
    prompt = RAG_PROMPT.format(
        chat_history=chat_history_str,
        context=context,
        question=standalone_question
    )
    resp = llm.invoke(prompt).content

    # Update memory (on stocke la question originale, et la réponse)
    history.append({"role": "user", "content": question})
    history.append({"role": "assistant", "content": resp})
    return resp


In [30]:
history = []
print(rag_chat(vectorstore_clean, "Parle-moi des railway communications.", history, k=5))

print(rag_chat(vectorstore_clean, "Et quelles sont les limites mentionnées précédement ?", history, k=5))


Les communications ferroviaires utilisent des technologies comme les ondes millimétriques (mmWave) pour la transmission de données. Les surfaces intelligentes réconfigurables (RIS) sont proposées comme solution prometteuse pour surmonter les problèmes de blocage des signaux mmWave, notamment dans les environnements ferroviaires. Elles permettent d'optimiser la propagation des signaux, notamment pour les trains à grande vitesse, en modulant les conditions du canal. Des recherches récentes explorent leur application dans ce domaine, avec des perspectives pour améliorer la connectivité et la fiabilité des réseaux ferroviaires.
Les communications ferroviaires mentionnées font face à des limites liées à la sensibilité des signaux mmWave au blocage, en raison de leur forte perte de pénétration, ce qui réduit leur couverture. Les surfaces intelligentes réconfigurables (RIS) sont présentées comme une solution prometteuse pour pallier ces problèmes en améliorant l'efficacité et la portée des co

##Exercice 6- nouveaux outils

## Récupération et résumé d’un document complet

Ce code permet de récupérer un document complet à partir de la base vectorielle et d’en produire un résumé.

Il regroupe tous les chunks appartenant à un même fichier source (`metadata.source`) afin de reconstruire
le contenu du document d’origine.

Concrètement :
- les chunks correspondant à un même fichier sont récupérés depuis ChromaDB via un filtre sur les métadonnées,
- les chunks sont triés selon leur `chunk_id` pour respecter l’ordre du document,
- le préfixe `"passage: "` ajouté lors de l’indexation est supprimé,
- les chunks sont concaténés pour reconstituer le texte complet du document,
- un prompt de résumé est appliqué au texte reconstitué,
- le modèle de langage génère un résumé structuré sous forme de puces avec des mots-clés finaux.

Ce mécanisme constitue un nouvel outil permettant de produire un résumé global d’un document
à partir des données indexées dans le système RAG.


In [31]:
from typing import Optional

def get_full_document_by_source(vectorstore: Chroma, source_path: str, max_chunks: int = 200) -> str:
    """
    Récupère (approximativement) un document complet en regroupant les chunks
    ayant metadata.source == source_path.
    """
    # Accès bas niveau à la collection Chroma
    coll = vectorstore._collection

    # where filtre sur metadata
    res = coll.get(
        where={"source": source_path},
        include=["documents", "metadatas"]
    )

    docs = res.get("documents", [])
    metas = res.get("metadatas", [])
    # Tri par chunk_id si présent
    paired = list(zip(docs, metas))
    paired.sort(key=lambda x: x[1].get("chunk_id", 0) if isinstance(x[1], dict) else 0)

    # dépréfixe "passage: "
    joined = "\n\n".join([d.replace("passage: ", "").strip() for d, _ in paired[:max_chunks]])
    return joined.strip()

SUMMARY_PROMPT = PromptTemplate(
    input_variables=["document"],
    template=(
        "Résume le document suivant en français.\n"
        "- Fais 8 à 12 puces.\n"
        "- Termine par 3 mots-clés.\n\n"
        "DOCUMENT:\n{document}\n"
    )
)

def summarize_source(vectorstore: Chroma, source_path: str) -> str:
    full_text = get_full_document_by_source(vectorstore, source_path)
    if not full_text:
        return "Document introuvable pour cette source."
    prompt = SUMMARY_PROMPT.format(document=full_text[:12000])  # garde-fou longueur
    return llm.invoke(prompt).content


In [32]:
results = retrieve_documents(vectorstore_clean, "railway communications", k=1)
src = results[0][0].metadata.get("source")
print("Source choisie:", src)
print(summarize_source(vectorstore_clean, src))


Source choisie: data/latex/Reconfigurable Intelligent Surface Assisted Railway Communications A survey.tex
- **Contexte** : La demande croissante de passagers et de débits de données élevés pour les technologies comme le streaming vidéo et l’IoT pousse à l’exploration des fréquences millimétriques (mmWave) pour les communications ferroviaires.  
- **Défis** : Les mmWave sont sensibles aux obstacles et souffrent de perte de propagation élevée, limitant leur couverture. Les surfaces intelligentes réconfigurables (RIS) offrent une solution prometteuse.  
- **RIS : définition** : Structure électromagnétique réconfigurable qui transforme le canal de propagation en environnement radio programmable, améliorant la qualité du signal et la couverture.  
- **Types de RIS** : Passif (réflexion sans amplification), actif (avec amplification), et hybride (combinaison des deux) pour optimiser l’énergie et les performances.  
- **Optimisation** : Algorithmes comme la transformation de dualité lagrangi

#Etape 3 - IHM

## Interface utilisateur Gradio

Ce code met en place une interface utilisateur graphique permettant d’interagir avec le chatbot RAG.

L’interface est construite avec Gradio et contient :
- une zone de discussion affichant les échanges utilisateur / assistant,
- un champ de saisie pour la question utilisateur,
- un curseur permettant de choisir le nombre de documents récupérés (`top-k`),
- un bouton pour envoyer un message,
- un bouton pour réinitialiser la conversation.

Le composant `gr.State` est utilisé pour stocker l’historique de la conversation.
Cet historique est transmis à chaque appel afin de conserver la mémoire entre les messages.

À chaque interaction :
- la question utilisateur est envoyée à la fonction `rag_chat`,
- la réponse générée est ajoutée à l’historique,
- l’ensemble des échanges est affiché dans le composant `Chatbot`.

Le bouton *Reset* permet de vider l’historique et de recommencer une nouvelle conversation.
L’option `share=True` rend l’interface accessible via une URL publique.


In [33]:
!pip install -q gradio


In [34]:
import gradio as gr

def gradio_chat(user_message, k, state):
    #historique
    if state is None:
        state = []

    answer = rag_chat(vectorstore_clean, user_message, state, k=int(k))


    pairs = []
    for i in range(0, len(state), 2):
        u = state[i]["content"]
        a = state[i+1]["content"]
        pairs.append((u, a))

    return pairs, state

def reset_chat():
    return [], []

with gr.Blocks() as demo:
    gr.Markdown("# TP RAG — Chatbot (Chroma + multilingual-e5-base + Qwen3 via Ollama)")
    k = gr.Slider(1, 10, value=5, step=1, label="Top-K documents")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Votre question")

    state = gr.State([])

    with gr.Row():
        send = gr.Button("Envoyer")
        clear = gr.Button("Reset")

    send.click(fn=gradio_chat, inputs=[msg, k, state], outputs=[chatbot, state])
    clear.click(fn=reset_chat, inputs=[], outputs=[chatbot, state])

demo.launch(share=True, debug=True)


/tmp/ipython-input-3218206157.py:25: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
/tmp/ipython-input-3218206157.py:25: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://30369d68c4e395f533.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://30369d68c4e395f533.gradio.live
